In [2]:
import pandas as pd
import requests

# Configuration
API_KEY = "603eafd117ce29.12741264"  # Your EODHD API key
tickers = [
    "AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "BRK-B", "AVGO", "TSLA", "LLY", 
    "WMT", "JPM", "V", "MA", "XOM", "COST", "UNH", "HD", "PG", "JNJ", "ABBV", "CRM", 
    "BAC", "ORCL", "MRK", "CVX", "WFC", "KO", "CSCO", "ACN"
]

# Load your CSV
df = pd.read_csv('data/merged_pe_roe.csv')

# Ensure dates are in 'YYYY-MM-DD' string format (matching API keys)
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

# Replace 'BRK.B' with 'BRK-B' in the DataFrame for consistency
df['Ticker'] = df['Ticker'].replace('BRK.B', 'BRK-B')

# Replace 'BRK.B' with 'BRK-B' in the tickers list for API calls
tickers = [ticker.replace('BRK.B', 'BRK-B') for ticker in tickers]

# Function to fetch filing dates from the Fundamental API
def fetch_filing_dates(tickers):
    filing_date_mapping = {}
    for ticker in tickers:
        try:
            # Use the corrected ticker format with .US suffix
            url = f"https://eodhd.com/api/fundamentals/{ticker}.US?api_token={API_KEY}&fmt=json"
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for failed requests
            data = response.json()
            # Navigate to quarterly balance sheet data
            quarterly = data.get('Financials', {}).get('Balance_Sheet', {}).get('quarterly', {})
            for date_key, report in quarterly.items():
                filing_date = report.get('filing_date')
                if filing_date:
                    filing_date_mapping[(ticker, date_key)] = filing_date
                else:
                    print(f"No filing_date for {ticker} on {date_key}")
        except requests.RequestException as e:
            print(f"Error fetching data for {ticker}: {e}")
    return filing_date_mapping

# Fetch filing dates
filing_date_mapping = fetch_filing_dates(tickers)

# Add Reported_Date column
df['Reported_Date'] = df.apply(
    lambda row: filing_date_mapping.get((row['Ticker'], row['Date']), None), axis=1
)

# Save the updated CSV
output_file = 'data/merged_pe_roe_updated.csv'
df.to_csv(output_file, index=False)

print(f"Updated CSV saved as '{output_file}' with 'Reported_Date' added.")

No filing_date for META on 2010-12-31
No filing_date for AVGO on 2007-10-31
No filing_date for UNH on 2006-09-30
No filing_date for UNH on 2006-06-30
No filing_date for UNH on 2001-12-31
No filing_date for JNJ on 1998-12-31
No filing_date for ABBV on 2010-12-31
No filing_date for CRM on 2003-01-31
No filing_date for ACN on 2001-08-31
No filing_date for ACN on 2001-05-31
No filing_date for ACN on 2001-02-28
No filing_date for ACN on 2000-08-31
Updated CSV saved as 'data/merged_pe_roe_updated.csv' with 'Reported_Date' added.
